# Análise de Dados - Olist

Este jupyter notebook apresenta uma análise geral dos dados da plataforma de vendas Olist, organizando e visualizando informações sobre clientes, produtos e vendas.

### 1. Importação das Bibliotecas e Leitura dos Dados

Carregamos os arquivos CSV para os respectivos DataFrames.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

caminho_arquivos = {
    "clientes": "dados/olist_customers_dataset.csv",
    "geolocalizacao": "dados/olist_geolocation_dataset.csv",
    "itens_pedidos": "dados/olist_order_items_dataset.csv",
    "pagamentos_pedidos": "dados/olist_order_payments_dataset.csv",
    "avaliacoes_pedidos": "dados/olist_order_reviews_dataset.csv",
    "pedidos": "dados/olist_orders_dataset.csv",
    "produtos": "dados/olist_products_dataset.csv",
    "vendedores": "dados/olist_sellers_dataset.csv",
    "traducao_categorias_produtos": "dados/product_category_name_translation.csv",
}

dataframes = {nome: pd.read_csv(caminho) for nome, caminho in caminho_arquivos.items()}

clientes_df = dataframes["clientes"]
geolocalizacao_df = dataframes["geolocalizacao"]
itens_pedidos_df = dataframes["itens_pedidos"]
pagamentos_pedidos_df = dataframes["pagamentos_pedidos"]
avaliacoes_pedidos_df = dataframes["avaliacoes_pedidos"]
pedidos_df = dataframes["pedidos"]
produtos_df = dataframes["produtos"]
vendedores_df = dataframes["vendedores"]
traducao_categorias_produtos_df = dataframes["traducao_categorias_produtos"]

### 2. Pré-visualização dos Dados

Exibimos uma amostra de cada DataFrame para entender sua estrutura e conteúdo.

In [2]:
clientes_df.sample(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
91674,8f08459a3c56d498990baa07ec4fe68d,f1a325803dd4c341c5a946ac84c71973,13185,hortolandia,SP
68866,09bd9c343926f0210c6a4dd9226c9246,5cffd635d8749c43510edef193be34c8,5717,sao paulo,SP


In [3]:
geolocalizacao_df.sample(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
13824,1419,-23.567658,-46.651142,são paulo,SP
525997,29055,-20.293655,-40.294587,vitoria,ES


In [4]:
itens_pedidos_df.sample(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
6592,0ee8a265feb5028eef234263bd7f5c23,1,eebbed5ed3b134eceb717496c47652ba,d1c281d3ae149232351cd8c8cc885f0d,2017-05-09 22:02:30,99.99,20.78
57143,820dae5229a9c2f6d7cfcf9e327c46a7,1,0d9374142ba86a7c22357ea192bcfde6,276677b5d08786d5dce7c2149dcce48b,2018-05-04 15:31:11,63.90,13.59


In [5]:
pagamentos_pedidos_df.sample(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
25674,577b75217094b43a3e032b217d67f70f,1,credit_card,2,111.67
92229,2e175b759e9d426f83e9b55b6a0661b2,1,boleto,1,36.78


In [6]:
avaliacoes_pedidos_df.sample(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
54076,e72fea66d7876d637f7c999bd1648d68,54e5702667dea09e5e5c511966be57f2,5,NaN,Está de parabéns.,2017-07-14 00:00:00,2017-07-17 10:53:29
2856,bc2f39cf3d3d57fd472587417c884056,7b214cd39f1ef2aaeb78dbbffc63439d,5,NaN,NaN,2018-04-26 00:00:00,2018-04-27 12:15:30


In [7]:
pedidos_df.sample(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
16323,bdbe430ba78a7cff2db4ac139372d5b5,04edcf9c41ea594087f93171236b28e2,delivered,2017-12-06 00:04:49,2017-12-06 00:12:42,2017-12-08 21:22:26,2017-12-27 21:25:03,2018-01-08 00:00:00
97591,9b78531e68513dbdcebdb2458f3929e1,efac88cd236a885f89eb0e3c60e1c0cf,delivered,2017-08-15 11:18:52,2017-08-17 03:05:51,2017-08-17 13:15:09,2017-08-21 21:59:50,2017-09-04 00:00:00


In [8]:
produtos_df.sample(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
12933,da721f960c721a863e6bd3ccf8ebb4de,informatica_acessorios,33.0,70.0,1.0,180.0,60.0,11.0,11.0
14933,a6b4a230f7ca939eb68485292f8b3389,relogios_presentes,42.0,713.0,3.0,600.0,19.0,9.0,15.0


In [9]:
vendedores_df.sample(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
746,9da1ca0bef29ee170e541927a5aeabdc,15500,votuporanga,SP
180,8b2492b7ff4468839593fafeed55bad9,24451,sao goncalo,RJ
1572,1c129092bf23f28a5930387c980c0dfc,2972,sao paulo,SP
1898,9fc70243fb26800cba56e8f014797004,30270,belo horizonte,MG
2575,d4e12e7884759a14fa0f5f896c791cae,2242,sao paulo,SP


In [10]:
traducao_categorias_produtos_df.sample(2)

,product_category_name,product_category_name_english
64,cds_dvds_musicais,cds_dvds_musicals
41,instrumentos_musicais,musical_instruments


In [11]:
produtos_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


### 3. Identificação de Valores Ausentes

Identificamos valores ausentes em cada coluna dos DataFrames e organizamos os resultados em uma tabela consolidada para melhor visualização.

In [12]:
resumo = []

for nome, df in dataframes.items():
    valores_ausentes = df.isnull().sum() 
    df_resumo = pd.DataFrame({
        "DataFrame": [nome] * len(valores_ausentes),
        "Coluna": valores_ausentes.index,
        "Valores ausentes": valores_ausentes.values
    })
    resumo.append(df_resumo)
    
df_resumo_final = pd.concat(resumo, ignore_index=True)
df_resumo_final


,DataFrame,Coluna,Valores ausentes
0,clientes,customer_id,0
1,clientes,customer_unique_id,0
2,clientes,customer_zip_code_prefix,0
3,clientes,customer_city,0
4,clientes,customer_state,0
5,geolocalizacao,geolocation_zip_code_prefix,0
6,geolocalizacao,geolocation_lat,0
7,geolocalizacao,geolocation_lng,0
8,geolocalizacao,geolocation_city,0
9,geolocalizacao,geolocation_state,0


### 4. Tratamento de Dados

#### 4.1 Substituição de Valores Ausentes

In [13]:
avaliacoes_pedidos_df['review_comment_message'] = avaliacoes_pedidos_df['review_comment_message'].fillna('Sem Comentário')
avaliacoes_pedidos_df['review_comment_title'] = avaliacoes_pedidos_df['review_comment_title'].fillna('Sem Título')

In [14]:
# preenche valores ausentes com a média nas colunas numéricas do DataFrame 'produtos_df'
colunas_numericas = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

produtos_df[colunas_numericas] = produtos_df[colunas_numericas].apply(lambda col: col.fillna(col.mean()))

### 5. Análise Exploratória de Dados (EDA)

Exploramos os dados para obter _insights_ sobre tempos de entrega, vendas por categoria e vendas por localização geográfica.

#### 5.1 Tempo Médio de Entrega

Calculamos o tempo médio de entrega com base nas datas de compra e entrega ao cliente:

In [15]:
pedidos_df['order_delivered_customer_date'] = pd.to_datetime(pedidos_df['order_delivered_customer_date'])
pedidos_df['order_purchase_timestamp'] = pd.to_datetime(pedidos_df['order_purchase_timestamp'])
pedidos_df['delivery_time'] = (pedidos_df['order_delivered_customer_date'] - pedidos_df['order_purchase_timestamp']).dt.days

tempo_medio_entrega = pedidos_df['delivery_time'].mean()
tempo_medio_entrega = round(tempo_medio_entrega, 2)
print(f"Tempo médio de entrega: {tempo_medio_entrega} dias")

Tempo médio de entrega: 12.09 dias


#### 5.2 Vendas por Categoria de Produto

Analisamos a frequência de vendas por categoria de produto, integrando os dados de itens de pedidos e produtos.

In [16]:
itens_produtos = pd.merge(itens_pedidos_df, produtos_df, on='product_id', how='inner')

vendas_por_categoria = itens_produtos['product_category_name'].value_counts()

vendas_por_categoria_df = pd.DataFrame(vendas_por_categoria).reset_index()
vendas_por_categoria_df.columns = ['Categoria de Produto', 'Frequência de Vendas']

vendas_por_categoria_df


,Categoria de Produto,Frequência de Vendas
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
...,...,...
68,cds_dvds_musicais,14
69,la_cuisine,14
70,pc_gamer,9
71,fashion_roupa_infanto_juvenil,8


#### 5.3 Vendas por Localização Geográfica

Exploramos a distribuição das vendas com base nas informações de localização de clientes e geolocalização.

##### Por estado:

In [17]:
vendas_geolocalizacao = pd.merge(clientes_df, geolocalizacao_df, 
                                  left_on='customer_zip_code_prefix', 
                                  right_on='geolocation_zip_code_prefix', 
                                  how='inner')

vendas_por_estado = vendas_geolocalizacao['geolocation_state'].value_counts()

vendas_por_estado_df = pd.DataFrame(vendas_por_estado).reset_index()
vendas_por_estado_df.columns = ['Estado', 'Frequência de Vendas']

vendas_por_estado_df


,Estado,Frequência de Vendas
0,SP,5620430
1,RJ,3015690
2,MG,2878728
3,RS,805370
4,PR,626021
5,SC,538638
6,BA,365875
7,ES,316654
8,GO,133146
9,MT,122395


#### Por cidade:

In [18]:
vendas_por_cidade = vendas_geolocalizacao['geolocation_city'].value_counts()

vendas_por_cidade_df = pd.DataFrame(vendas_por_cidade).reset_index()
vendas_por_cidade_df.columns = ['Cidade', 'Frequência de Vendas']

vendas_por_cidade_df


,Cidade,Frequência de Vendas
0,rio de janeiro,1913913
1,sao paulo,1164470
2,belo horizonte,737556
3,niteroi,393175
4,curitiba,255731
...,...,...
5807,josé boiteux,1
5808,pinhal da serra,1
5809,vitorinos,1
5810,vitorinos - alto rio doce,1
